In [1]:
from os import path
import sys

# Importing Python modules

From the earlier notebooks in this tutorial, hopefully it should be clear that in order to import something into Python, that the objects need to be on the library path.

In this notebook, the import paths will be updated (in a rather nasty way, but at least explicit), and a module imported.

The modules sit in a folder next to the notebooks folder.

In [2]:
!tree -I __pycache__ ../non_buildable

../non_buildable
└── number_returns
    ├── number_returns
    │   ├── gimmes.py
    │   └── gimme_strs.py
    └── tests
        ├── test_gimmes.py
        └── test_gimme_strs.py

3 directories, 4 files


In [3]:
!cd ../non_buildable/number_returns/; python -m pytest
# these tests should succeed

============================= test session starts ==============================
platform linux -- Python 3.10.4, pytest-7.1.1, pluggy-1.0.0
rootdir: /home/jsnagi/proj/github/nagi49000/tutorial-memory-refs/python/non_buildable/number_returns
collected 3 items                                                              

tests/test_gimme_strs.py .                                               [ 33%]
tests/test_gimmes.py ..                                                  [100%]

============================== 3 passed in 0.01s ===============================


Due to specifying the folder location, the current working directory that is implicitly in the sys.path means that all imports, relative to ".", which in the case above was "../non_buildable/number_returns/", all work. This does mean that running anywhere else will mean the relative imports fail.

In [4]:
!cd ../non_buildable/; python -m pytest
# these tests should fail

============================= test session starts ==============================
platform linux -- Python 3.10.4, pytest-7.1.1, pluggy-1.0.0
rootdir: /home/jsnagi/proj/github/nagi49000/tutorial-memory-refs/python/non_buildable
collected 0 items / 2 errors                                                   

==================================== ERRORS ====================================
___________ ERROR collecting number_returns/tests/test_gimme_strs.py ___________
ImportError while importing test module '/home/jsnagi/proj/github/nagi49000/tutorial-memory-refs/python/non_buildable/number_returns/tests/test_gimme_strs.py'.
Hint: make sure your test modules/packages have valid Python names.
Traceback:
/home/jsnagi/miniconda3/envs/tutorial-mem-refs/lib/python3.10/importlib/__init__.py:126: in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
number_returns/tests/test_gimme_strs.py:1: in <module>
    from number_returns.gimme_strs import gimme5str
E   ModuleNotF

In [5]:
!cd ../non_buildable/number_returns/tests/; python -m pytest
# these tests should fail

============================= test session starts ==============================
platform linux -- Python 3.10.4, pytest-7.1.1, pluggy-1.0.0
rootdir: /home/jsnagi/proj/github/nagi49000/tutorial-memory-refs/python/non_buildable/number_returns/tests
collected 0 items / 2 errors                                                   

==================================== ERRORS ====================================
_____________________ ERROR collecting test_gimme_strs.py ______________________
ImportError while importing test module '/home/jsnagi/proj/github/nagi49000/tutorial-memory-refs/python/non_buildable/number_returns/tests/test_gimme_strs.py'.
Hint: make sure your test modules/packages have valid Python names.
Traceback:
/home/jsnagi/miniconda3/envs/tutorial-mem-refs/lib/python3.10/importlib/__init__.py:126: in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
test_gimme_strs.py:1: in <module>
    from number_returns.gimme_strs import gimme5str
E   ModuleNotF

This can be fixed by having the tests directory having an \_\_init\_\_.py. This file will be picked up by pytest when it tries to import the tests module; pytest will perform its test file discovery, and attempt to import the relevant modules. Recall that an import on the modules runs the relevant init, which in this case will add the required path to the sys.path

In [6]:
!tree -I __pycache__ ../non_buildable_2

../non_buildable_2
└── number_returns
    ├── number_returns
    │   ├── gimmes.py
    │   └── gimme_strs.py
    └── tests
        ├── __init__.py
        ├── test_gimmes.py
        └── test_gimme_strs.py

3 directories, 5 files


In [7]:
!cat ../non_buildable_2/number_returns/tests/__init__.py

import os
import sys

sys.path.append(os.path.join(".."))


In [8]:
!cd ../non_buildable_2/; python -m pytest
# these tests should succeed

============================= test session starts ==============================
platform linux -- Python 3.10.4, pytest-7.1.1, pluggy-1.0.0
rootdir: /home/jsnagi/proj/github/nagi49000/tutorial-memory-refs/python/non_buildable_2
collected 3 items                                                              

number_returns/tests/test_gimme_strs.py .                                [ 33%]
number_returns/tests/test_gimmes.py ..                                   [100%]

============================== 3 passed in 0.01s ===============================


In [9]:
!cd ../non_buildable_2/number_returns/tests/; python -m pytest
# these tests should succeed

============================= test session starts ==============================
platform linux -- Python 3.10.4, pytest-7.1.1, pluggy-1.0.0
rootdir: /home/jsnagi/proj/github/nagi49000/tutorial-memory-refs/python/non_buildable_2/number_returns/tests
collected 3 items                                                              

test_gimme_strs.py .                                                     [ 33%]
test_gimmes.py ..                                                        [100%]

============================== 3 passed in 0.01s ===============================


This now starts to move towards a buildable structure where there can be multiple libs and modules sitting next to eachother, and the tests can be run using a simple command at a top level above the library folder structure, which can be flexibly chosen (and is independent of the code). This is the minimal starting point one would want to have before being able to integrate with any kind of DevOps.